In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
import pandas as pd
import numpy as np
from collections import defaultdict
from itertools import combinations
from itertools import chain
from collections import namedtuple
import pickle
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
import os.path
import shutil
import inspect
from nn_lib import *
from train_utils import *
from cifar100_utils import *
from ft_utils import *
from icarl_utils import *
from ewc_utils import *
from myil import *
import matplotlib.pyplot as plt
import json
from pprint import pprint
from copy import copy
from sklearn.metrics import confusion_matrix
import seaborn as sns
np.random.seed(1997)
%load_ext autoreload
%autoreload 2

In [4]:
data_dict=load_cifar100_data()

In [5]:
data_dict_total=data_dict

In [52]:
hyper_params={'beta':1e-5,'initial_lr':2,'train_batch_size':128,'test_batch_size':128,'lr_reduction_rate':5,'lr_reduction_epoch':[49,63],
              'exemplar_set_size':50,'final_train_epochs':5,'train_method':'train_distillation_and_ground_truth','sample_weight':'half','lambda':10,'se':True,'primary_exemplar':'svm_exemplar'}
fixed_params={'net_type':'ResNet32','random_seed':1997,'total_num_classes':100,'base_dir':'./cifar100_icarl_debug_temp','class_batch_size':10,'use_theoretical_mean':True}

In [53]:
tf_tensors,tf_variables,tf_networks=build_graph_icarl(hyper_params,fixed_params)

In [54]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess=tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [61]:
fit_icarl(tf_tensors,tf_variables,tf_networks,fixed_params,hyper_params,data_dict,sess,resume=False,
        save_session=False,save_session_freq=1,save_params=True,evaluation_metric='top1_accuracy',save_history=True,
        num_epochs=1,num_iterations=1,verbose=2,print_freq=1,pretrain_evaluation=1,override_warning=True)

{'base_dir': './cifar100_icarl_debug_temp',
 'class_batch_size': 10,
 'net_type': 'ResNet32',
 'random_seed': 1997,
 'total_num_classes': 100,
 'use_theoretical_mean': True}
{'beta': 1e-05,
 'exemplar_set_size': 50,
 'final_train_epochs': 5,
 'initial_lr': 2,
 'lambda': 10,
 'lr_reduction_epoch': [49, 63],
 'lr_reduction_rate': 5,
 'primary_exemplar': 'svm_exemplar',
 'sample_weight': 'half',
 'se': True,
 'test_batch_size': 128,
 'train_batch_size': 128,
 'train_method': 'train_distillation_and_ground_truth'}
./cifar100_icarl_debug_temp already exists, override?y
===========Iteration 1=============
Using classes [88, 77, 36, 54, 92, 4, 11, 24, 71, 31]
Class batch pretrain evaluation (plain method)
	Train loss: 0.792616
	Train class loss: 0.777767
	Train reg loss: 0.067708
	Validation loss: 0.793290
	Validation class loss: 0.778441
	Validation reg loss: 0.014849
	Top1 train accuracy: 0.067708
	Top5 train accuracy: 0.100160
	Top1 validation accuracy: 0.074777
	Top5 validation accuracy: 

In [25]:
list(tf_networks['train_network'].prev_variables.keys())

['conv_1/W',
 'resconv1_2a/W',
 'resconv2_2a/W',
 'resconv1_2b/W',
 'resconv2_2b/W',
 'resconv1_2c/W',
 'resconv2_2c/W',
 'resconv1_2d/W',
 'resconv2_2d/W',
 'resconv1_2e/W',
 'resconv2_2e/W',
 'resconv1_3a/W',
 'resconv2_3a/W',
 'resconv1_3b/W',
 'resconv2_3b/W',
 'resconv1_3c/W',
 'resconv2_3c/W',
 'resconv1_3d/W',
 'resconv2_3d/W',
 'resconv1_3e/W',
 'resconv2_3e/W',
 'resconv1_4a/W',
 'resconv2_4a/W',
 'resconv1_4b/W',
 'resconv2_4b/W',
 'resconv1_4c/W',
 'resconv2_4c/W',
 'resconv1_4d/W',
 'resconv2_4d/W',
 'resconv1_4e/W',
 'resconv2_4e/W',
 'fc/W']

In [30]:
list(tf_networks['train_network'].fisher_variables.keys())

['conv_1/W',
 'resconv1_2a/W',
 'resconv2_2a/W',
 'resconv1_2b/W',
 'resconv2_2b/W',
 'resconv1_2c/W',
 'resconv2_2c/W',
 'resconv1_2d/W',
 'resconv2_2d/W',
 'resconv1_2e/W',
 'resconv2_2e/W',
 'resconv1_3a/W',
 'resconv2_3a/W',
 'resconv1_3b/W',
 'resconv2_3b/W',
 'resconv1_3c/W',
 'resconv2_3c/W',
 'resconv1_3d/W',
 'resconv2_3d/W',
 'resconv1_3e/W',
 'resconv2_3e/W',
 'resconv1_4a/W',
 'resconv2_4a/W',
 'resconv1_4b/W',
 'resconv2_4b/W',
 'resconv1_4c/W',
 'resconv2_4c/W',
 'resconv1_4d/W',
 'resconv2_4d/W',
 'resconv1_4e/W',
 'resconv2_4e/W',
 'fc/W']

In [60]:
list(tf_networks['train_network'].tf_tensors.keys())

['input',
 'conv_1',
 'batch_norm_1',
 'residual_block_2a',
 'residual_block_2b',
 'residual_block_2c',
 'residual_block_2d',
 'residual_block_2e',
 'residual_block_3a',
 'residual_block_3b',
 'residual_block_3c',
 'residual_block_3d',
 'residual_block_3e',
 'residual_block_4a',
 'residual_block_4b',
 'residual_block_4c',
 'residual_block_4d',
 'residual_block_4e',
 'pool_last',
 'fc',
 'l2_loss',
 'ewc_loss']

In [80]:
sess.run((tf_networks['train_network'].regularizable_variables['conv_1/W']))

FailedPreconditionError: Attempting to use uninitialized value ResNet32_train/conv_1/W
	 [[Node: ResNet32_train/conv_1/W/_0 = _Send[T=DT_FLOAT, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_4_ResNet32_train/conv_1/W", _device="/job:localhost/replica:0/task:0/gpu:0"](ResNet32_train/conv_1/W)]]
	 [[Node: ResNet32_train/conv_1/W/_1 = _Recv[_start_time=0, client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_4_ResNet32_train/conv_1/W", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

In [44]:
for k in tf_networks['train_network'].fisher_variables:
    print(np.max(sess.run(tf_networks['train_network'].fisher_variables[k])))

9.23201
1.79454
1.01642
0.311387
0.164839
0.249598
0.156045
0.108914
0.162977
0.232894
0.150115
0.107237
0.0674546
0.0800215
0.0609434
0.116719
0.0609143
0.033652
0.0624136
0.0180974
0.0440502
0.0358085
0.0585388
0.055075
0.0379839
0.0302096
0.0296525
0.150095
0.0558424
0.187442
0.0853254
0.727345


In [28]:
tf_networks['train_network'].fisher_variables

{'conv_1/W': <tf.Variable 'ResNet32_train/conv_1/W_fisher:0' shape=(3, 3, 3, 16) dtype=float32_ref>,
 'fc/W': <tf.Variable 'ResNet32_train/fc/W_fisher:0' shape=(1, 1, 64, 100) dtype=float32_ref>,
 'resconv1_2a/W': <tf.Variable 'ResNet32_train/resconv1_2a/W_fisher:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
 'resconv1_2b/W': <tf.Variable 'ResNet32_train/resconv1_2b/W_fisher:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
 'resconv1_2c/W': <tf.Variable 'ResNet32_train/resconv1_2c/W_fisher:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
 'resconv1_2d/W': <tf.Variable 'ResNet32_train/resconv1_2d/W_fisher:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
 'resconv1_2e/W': <tf.Variable 'ResNet32_train/resconv1_2e/W_fisher:0' shape=(3, 3, 16, 16) dtype=float32_ref>,
 'resconv1_3a/W': <tf.Variable 'ResNet32_train/resconv1_3a/W_fisher:0' shape=(3, 3, 16, 32) dtype=float32_ref>,
 'resconv1_3b/W': <tf.Variable 'ResNet32_train/resconv1_3b/W_fisher:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 'resconv1_3c/W': <t

In [4]:
tf.reset_default_graph()
net=ResNet('ResNet32','test',num_outputs=100,name='ResNet32_test',use_fisher=True)
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess=tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
tf.global_variables_initializer().run()